In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [6]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

model.save('digit_recognition_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5


d:\Software\Anaconda3\install\envs\pycharm\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8956 - loss: 0.3407 - val_accuracy: 0.9862 - val_loss: 0.0432
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9857 - loss: 0.0470 - val_accuracy: 0.9890 - val_loss: 0.0356
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9895 - loss: 0.0317 - val_accuracy: 0.9901 - val_loss: 0.0332
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9927 - loss: 0.0229 - val_accuracy: 0.9909 - val_loss: 0.0269
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9939 - loss: 0.0187 - val_accuracy: 0.9902 - val_loss: 0.0304


In [12]:
import numpy as np
import cv2
from PIL import ImageGrab
from tensorflow.keras.models import load_model

In [13]:
model = load_model('digit_recognition_model.h5')

In [27]:
def capture_screen(region):
    screen = ImageGrab.grab(bbox=region)
    # screen = Image.open("./image.png")
    return np.array(screen)


def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV) 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    processed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    return processed


def find_contours(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def extract_digits(image, contours):
    digit_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 20 and h < 100: 
            roi = image[y : y + h, x : x + w]
            digit_images.append((x, roi))

    digit_images = sorted(digit_images, key=lambda item: item[0])
    return [digit[1] for digit in digit_images]


def recognize_digit(image):
    resized = cv2.resize(image, (28, 28))
    normalized = resized / 255.0
    reshaped = normalized.reshape(1, 28, 28, 1)
    prediction = model.predict(reshaped)
    return np.argmax(prediction)


def recognize_digits(image):
    contours = find_contours(image)
    digit_images = extract_digits(image, contours)
    recognized_digits = [recognize_digit(digit) for digit in digit_images]
    return "".join(map(str, recognized_digits))

In [29]:
import pyautogui

In [39]:
def getDigital(region):
    screenshot = capture_screen(region)
    preprocessed_image = preprocess_image(screenshot)
    recognized_digits = recognize_digits(preprocessed_image)
    return recognize_digit


def draw_greater_than(start_x, start_y):
    pyautogui.moveTo(start_x, start_y)
    pyautogui.mouseDown()
    pyautogui.mouseDown()
    pyautogui.moveTo(start_x + 100, start_y - 100, duration=0.1)  # 右上
    pyautogui.moveTo(start_x + 100, start_y + 100, duration=0.1)  # 右下
    pyautogui.moveTo(start_x, start_y, duration=0.3)  # 左上
    pyautogui.mouseUp()


def draw_less_than(start_x, start_y):
    pyautogui.moveTo(start_x, start_y)
    pyautogui.mouseDown()
    pyautogui.mouseDown()
    pyautogui.moveTo(start_x - 100, start_y - 100, duration=0.1)  # 左上
    pyautogui.moveTo(start_x - 100, start_y + 100, duration=0.1)  # 左下
    pyautogui.moveTo(start_x, start_y, duration=0.3)  # 右上
    pyautogui.mouseUp()

In [48]:
while(True):
    region_1 = (247, 326, 292, 386)
    region_2 = (399, 323, 474, 395)
    num_1 = recognize_digit(region_1)
    num_2 = recognize_digit(region_2)
    print(num_1)
    print(num_2)
    if num_1 > num_2:
        draw_greater_than(333, 482)
    else:
        draw_less_than(333, 482)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.